In [1]:
# Mount your google drive. 
# Use this to save your PyTorch model for submission
from google.colab import drive
drive.mount('/content/gdrive')
!mkdir /content/gdrive/Team\ Drives/cis530
#Test drive access. 
#You should have a test.txt in your Google drive
with open('/content/gdrive/Team Drives/cis530/test.txt', 'w') as f:
  f.write('This is a test file!')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
mkdir: cannot create directory ‘/content/gdrive/Team Drives/cis530’: File exists


In [0]:
from nltk.corpus import conll2002
from nltk.corpus import cess_esp as cess

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Perceptron
from sklearn.metrics import precision_recall_fscore_support

from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.svm import LinearSVC
import pickle

In [0]:
def isApostrophePresent(word):
    if "'" in word:
        return True
    return False

In [0]:
def getfeats(word, postag, o, no_of_features = None):
    """ This takes the word in question and
    the offset with respect to the instance
    word """
    o = str(o)
    
    features = [
        
        (o + 'word', word),
        # TODO: add more features here.
        (o + 'word.isupper', any(letter.isupper() for letter in word)),
        (o + 'word.len', len(word) ),
        (o + 'word.isdigit', any(letter.isdigit() for letter in word))
        
#         (o + 'isApostrophePresent', isApostrophePresent(word))
    ]
    return features
  
    
    if( no_of_features != None ):
      return features
    else:
      return features[:no_of_features]

In [0]:
def word2features(sent, i , no_of_features = None):
    """ The function generates all features
    for the word at position i in the
    sentence."""
    features = []
    # the window around the token
    featlist = [('bias', 1.0)]
    features.extend(featlist)

    for o in [-1,0,1]:
        if i+o >= 0 and i+o < len(sent):
            word = sent[i+o][0]
#             postag = sent[i+o][1]
            postag = "unknown"
            featlist = getfeats(word, postag, o, no_of_features)
            features.extend(featlist)
        elif i+o<0:
            featlist = [('BOS', 1)]
            features.extend(featlist)
        else:
            featlist = [('EOS', 1)]
            features.extend(featlist)    
    return dict(features)

In [0]:
def runClassifier(model,no_of_features):
    #Gathers the data

    #Reports the performance of train/dev/test
    train_feats = []
    train_labels = []

    for sent in train_sents:
        for i in range(len(sent)):
            feats = word2features(sent,i,no_of_features)
            
            train_feats.append(feats)
            train_labels.append(sent[i][-1])

    vectorizer = DictVectorizer()
    X_train = vectorizer.fit_transform(train_feats)

        
    #Fit the data to the model
    model.fit(X_train, train_labels)


    #Training Data
    y_train_pred = model.predict(X_train)

    j = 0
    print("Writing to train_results.txt")
    # format is: word gold pred
    with open("/content/gdrive/Team Drives/cis530/results/train_results.txt", "w") as out:
        for sent in train_sents: 
            for i in range(len(sent)):
                word = sent[i][0]
                gold = sent[i][-1]
                pred = y_train_pred[j]
                j += 1
                out.write("{}\t{}\t{}\n".format(word,gold,pred))
        out.write("\n")

    #Dev Data
    dev_feats = []
    dev_labels = []

    # switch to test_sents for your final results
    for sent in dev_sents:
        for i in range(len(sent)):
            feats = word2features(sent,i)
            dev_feats.append(feats)
            dev_labels.append(sent[i][-1])

    X_dev = vectorizer.transform(dev_feats)
    y_dev_pred = model.predict(X_dev)

    j = 0
    print("Writing to dev_results.txt")
    # format is: word gold pred
    with open("/content/gdrive/Team Drives/cis530/results/dev_results.txt", "w") as out:
        for sent in dev_sents: 
            for i in range(len(sent)):
                word = sent[i][0]
                gold = sent[i][-1]
                pred = y_dev_pred[j]
                j += 1
                out.write("{}\t{}\t{}\n".format(word,gold,pred))
        out.write("\n")

    #Test Data
    test_feats = []
    test_labels = []

    # switch to test_sents for your final results
    for sent in test_sents:
        for i in range(len(sent)):
            feats = word2features(sent,i)
            test_feats.append(feats)
            test_labels.append(sent[i][-1])

    X_test = vectorizer.transform(test_feats)
    y_test_pred = model.predict(X_test)

    j = 0
    print("Writing to test_results.txt")
    # format is: word gold pred
    with open("/content/gdrive/Team Drives/cis530/results/test_results.txt", "w") as out:
        for sent in test_sents: 
            for i in range(len(sent)):
                word = sent[i][0]
                gold = sent[i][-1]
                pred = y_test_pred[j]
                j += 1
                out.write("{}\t{}\t{}\n".format(word,gold,pred))
        out.write("\n")
        
        
    #Evaluate training results
    print('Train Results for the model',model,' with ',no_of_features,' of features')
    
    with open('/content/gdrive/Team Drives/cis530/results/train_results.txt') as f:
      counts = evaluate(f)
    report(counts)
    
    #Evaluate Dev results
    print('Dev Results for the model',model,' with ',no_of_features,' of features')
    
    with open('/content/gdrive/Team Drives/cis530/results/dev_results.txt') as f:
      counts = evaluate(f)
    report(counts)
    
    #Evaluate test results
    print('Test Results for the model',model,' with ',no_of_features,' of features')
    
    with open('/content/gdrive/Team Drives/cis530/results/test_results.txt') as f:
      counts = evaluate(f)
    report(counts)

In [0]:

if __name__ == "__main__":
  
    # Load the training data
    file = open('/content/gdrive/Team Drives/cis530/ner_tagged_sentences/train_ner_tagged_sentences.pickle','rb')
    train_sents = pickle.load(file)
    file.close()
    
    print("train_sents len= ", len(train_sents))
      
    file = open('/content/gdrive/Team Drives/cis530/ner_tagged_sentences/dev_ner_tagged_sentences.pickle','rb')
    dev_sents = pickle.load(file)
    file.close()
    
    print("dev_sents len= ", len(dev_sents))
      
    file = open('/content/gdrive/Team Drives/cis530/ner_tagged_sentences/test_ner_tagged_sentences.pickle','rb')
    test_sents = pickle.load(file)
    file.close()
    
    print("test_sents len= ", len(test_sents))
    
    no_of_features = 4
    models = [LinearSVC(),LinearSVC(C=0.01),LinearSVC(C=10)]
    
    for model in models:
      for i in range(1,no_of_features+1,1):
        runClassifier(model,i)

train_sents len=  8300
dev_sents len=  4048
test_sents len=  3231


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Writing to train_results.txt
Writing to dev_results.txt
Writing to test_results.txt
Train Results for the model LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)  with  1  of features
processed 157843 tokens with 8675 phrases; found: 8942 phrases; correct: 8488.
accuracy:  99.67%; precision:  94.92%; recall:  97.84%; FB1:  96.36
            CALLE: precision:  80.58%; recall:  93.27%; FB1:  86.46  757
     CENTRO_SALUD: precision:  80.00%; recall: 100.00%; FB1:  88.89  5
CORREO_ELECTRONICO: precision:  91.55%; recall:  98.53%; FB1:  94.92  367
EDAD_SUJETO_ASISTENCIA: precision:  91.74%; recall:  96.04%; FB1:  93.84  847
FAMILIARES_SUJETO_ASISTENCIA: precision:  88.41%; recall:  83.82%; FB1:  86.05  164
           FECHAS: precision:  99.90%; recall:  99.49%; FB1:  99.70  983
         HOSPITAL: precision:  80.75%; recall:  92.

In [0]:
#Evaluation
#!/usr/bin/env python

# Python version of the evaluation script from CoNLL'00-

# Intentional differences:
# - accept any space as delimiter by default
# - optional file argument (default STDIN)
# - option to set boundary (-b argument)
# - LaTeX output (-l argument) not supported
# - raw tags (-r argument) not supported

import sys
import re

from collections import defaultdict, namedtuple

ANY_SPACE = '<SPACE>'

class FormatError(Exception):
    pass

Metrics = namedtuple('Metrics', 'tp fp fn prec rec fscore')

class EvalCounts(object):
    def __init__(self):
        self.correct_chunk = 0    # number of correctly identified chunks
        self.correct_tags = 0     # number of correct chunk tags
        self.found_correct = 0    # number of chunks in corpus
        self.found_guessed = 0    # number of identified chunks
        self.token_counter = 0    # token counter (ignores sentence breaks)

        # counts by type
        self.t_correct_chunk = defaultdict(int)
        self.t_found_correct = defaultdict(int)
        self.t_found_guessed = defaultdict(int)

def parse_args(argv):
    import argparse
    parser = argparse.ArgumentParser(
        description='evaluate tagging results using CoNLL criteria',
        formatter_class=argparse.ArgumentDefaultsHelpFormatter
    )
    arg = parser.add_argument
    arg('-b', '--boundary', metavar='STR', default='-X-',
        help='sentence boundary')
    arg('-d', '--delimiter', metavar='CHAR', default=ANY_SPACE,
        help='character delimiting items in input')
    arg('-o', '--otag', metavar='CHAR', default='O',
        help='alternative outside tag')
    arg('file', nargs='?', default=None)
    return parser.parse_args(argv)

def parse_tag(t):
    m = re.match(r'^([^-]*)-(.*)$', t)
    return m.groups() if m else (t, '')

def evaluate(iterable, options=None):
    if options is None:
        options = parse_args([])    # use defaults

    counts = EvalCounts()
    num_features = None       # number of features per line
    in_correct = False        # currently processed chunks is correct until now
    last_correct = 'O'        # previous chunk tag in corpus
    last_correct_type = ''    # type of previously identified chunk tag
    last_guessed = 'O'        # previously identified chunk tag
    last_guessed_type = ''    # type of previous chunk tag in corpus

    for line in iterable:
        line = line.rstrip('\r\n')

        if options.delimiter == ANY_SPACE:
            features = line.split()
        else:
            features = line.split(options.delimiter)

        if num_features is None:
            num_features = len(features)
        elif num_features != len(features) and len(features) != 0:
            raise FormatError('unexpected number of features: %d (%d)' %
                              (len(features), num_features))

        if len(features) == 0 or features[0] == options.boundary:
            features = [options.boundary, 'O', 'O']
        if len(features) < 3:
            raise FormatError('unexpected number of features in line %s' % line)

        guessed, guessed_type = parse_tag(features.pop())
        correct, correct_type = parse_tag(features.pop())
        first_item = features.pop(0)

        if first_item == options.boundary:
            guessed = 'O'

        end_correct = end_of_chunk(last_correct, correct,
                                   last_correct_type, correct_type)
        end_guessed = end_of_chunk(last_guessed, guessed,
                                   last_guessed_type, guessed_type)
        start_correct = start_of_chunk(last_correct, correct,
                                       last_correct_type, correct_type)
        start_guessed = start_of_chunk(last_guessed, guessed,
                                       last_guessed_type, guessed_type)

        if in_correct:
            if (end_correct and end_guessed and
                last_guessed_type == last_correct_type):
                in_correct = False
                counts.correct_chunk += 1
                counts.t_correct_chunk[last_correct_type] += 1
            elif (end_correct != end_guessed or guessed_type != correct_type):
                in_correct = False

        if start_correct and start_guessed and guessed_type == correct_type:
            in_correct = True

        if start_correct:
            counts.found_correct += 1
            counts.t_found_correct[correct_type] += 1
        if start_guessed:
            counts.found_guessed += 1
            counts.t_found_guessed[guessed_type] += 1
        if first_item != options.boundary:
            if correct == guessed and guessed_type == correct_type:
                counts.correct_tags += 1
            counts.token_counter += 1

        last_guessed = guessed
        last_correct = correct
        last_guessed_type = guessed_type
        last_correct_type = correct_type

    if in_correct:
        counts.correct_chunk += 1
        counts.t_correct_chunk[last_correct_type] += 1

    return counts

def uniq(iterable):
  seen = set()
  return [i for i in iterable if not (i in seen or seen.add(i))]

def calculate_metrics(correct, guessed, total):
    tp, fp, fn = correct, guessed-correct, total-correct
    p = 0 if tp + fp == 0 else 1.*tp / (tp + fp)
    r = 0 if tp + fn == 0 else 1.*tp / (tp + fn)
    f = 0 if p + r == 0 else 2 * p * r / (p + r)
    return Metrics(tp, fp, fn, p, r, f)

def metrics(counts):
    c = counts
    overall = calculate_metrics(
        c.correct_chunk, c.found_guessed, c.found_correct
    )
    by_type = {}
    for t in uniq(list(c.t_found_correct) + list(c.t_found_guessed)):
        by_type[t] = calculate_metrics(
            c.t_correct_chunk[t], c.t_found_guessed[t], c.t_found_correct[t]
        )
    return overall, by_type

def report(counts, out=None):
    if out is None:
        out = sys.stdout

    overall, by_type = metrics(counts)

    c = counts
    out.write('processed %d tokens with %d phrases; ' %
              (c.token_counter, c.found_correct))
    out.write('found: %d phrases; correct: %d.\n' %
              (c.found_guessed, c.correct_chunk))

    if c.token_counter > 0:
        out.write('accuracy: %6.2f%%; ' %
                  (100.*c.correct_tags/c.token_counter))
        out.write('precision: %6.2f%%; ' % (100.*overall.prec))
        out.write('recall: %6.2f%%; ' % (100.*overall.rec))
        out.write('FB1: %6.2f\n' % (100.*overall.fscore))

    for i, m in sorted(by_type.items()):
        out.write('%17s: ' % i)
        out.write('precision: %6.2f%%; ' % (100.*m.prec))
        out.write('recall: %6.2f%%; ' % (100.*m.rec))
        out.write('FB1: %6.2f  %d\n' % (100.*m.fscore, c.t_found_guessed[i]))

def end_of_chunk(prev_tag, tag, prev_type, type_):
    # check if a chunk ended between the previous and current word
    # arguments: previous and current chunk tags, previous and current types
    chunk_end = False

    if prev_tag == 'E': chunk_end = True
    if prev_tag == 'S': chunk_end = True

    if prev_tag == 'B' and tag == 'B': chunk_end = True
    if prev_tag == 'B' and tag == 'S': chunk_end = True
    if prev_tag == 'B' and tag == 'O': chunk_end = True
    if prev_tag == 'I' and tag == 'B': chunk_end = True
    if prev_tag == 'I' and tag == 'S': chunk_end = True
    if prev_tag == 'I' and tag == 'O': chunk_end = True

    if prev_tag != 'O' and prev_tag != '.' and prev_type != type_:
        chunk_end = True

    # these chunks are assumed to have length 1
    if prev_tag == ']': chunk_end = True
    if prev_tag == '[': chunk_end = True

    return chunk_end

def start_of_chunk(prev_tag, tag, prev_type, type_):
    # check if a chunk started between the previous and current word
    # arguments: previous and current chunk tags, previous and current types
    chunk_start = False

    if tag == 'B': chunk_start = True
    if tag == 'S': chunk_start = True

    if prev_tag == 'E' and tag == 'E': chunk_start = True
    if prev_tag == 'E' and tag == 'I': chunk_start = True
    if prev_tag == 'S' and tag == 'E': chunk_start = True
    if prev_tag == 'S' and tag == 'I': chunk_start = True
    if prev_tag == 'O' and tag == 'E': chunk_start = True
    if prev_tag == 'O' and tag == 'I': chunk_start = True

    if tag != 'O' and tag != '.' and prev_type != type_:
        chunk_start = True

    # these chunks are assumed to have length 1
    if tag == '[': chunk_start = True
    if tag == ']': chunk_start = True

    return chunk_start



In [8]:
#Evaluate training results

with open('/content/gdrive/Team Drives/cis530/results/train_results.txt') as f:
    counts = evaluate(f)
report(counts)

processed 157843 tokens with 8675 phrases; found: 8990 phrases; correct: 8336.
accuracy:  99.50%; precision:  92.73%; recall:  96.09%; FB1:  94.38
            CALLE: precision:  68.03%; recall:  83.64%; FB1:  75.03  804
     CENTRO_SALUD: precision:  50.00%; recall:  75.00%; FB1:  60.00  6
CORREO_ELECTRONICO: precision:  90.13%; recall:  99.12%; FB1:  94.41  375
EDAD_SUJETO_ASISTENCIA: precision:  90.86%; recall:  95.80%; FB1:  93.26  853
FAMILIARES_SUJETO_ASISTENCIA: precision:  85.71%; recall:  79.77%; FB1:  82.63  161
           FECHAS: precision:  99.38%; recall:  98.18%; FB1:  98.78  975
         HOSPITAL: precision:  71.89%; recall:  84.32%; FB1:  77.61  217
 ID_ASEGURAMIENTO: precision:  98.96%; recall: 100.00%; FB1:  99.48  288
ID_CONTACTO_ASISTENCIAL: precision: 100.00%; recall:  98.08%; FB1:  99.03  51
ID_SUJETO_ASISTENCIA: precision:  98.57%; recall:  97.88%; FB1:  98.22  421
ID_TITULACION_PERSONAL_SANITARIO: precision:  98.88%; recall:  99.72%; FB1:  99.29  356
      INSTIT

In [9]:
#Evaluate dev results

with open('/content/gdrive/Team Drives/cis530/results/dev_results.txt') as f:
    counts = evaluate(f)
report(counts)


processed 74003 tokens with 4332 phrases; found: 4733 phrases; correct: 2968.
accuracy:  96.85%; precision:  62.71%; recall:  68.51%; FB1:  65.48
            CALLE: precision:  14.31%; recall:  27.59%; FB1:  18.84  615
     CENTRO_SALUD: precision:   0.00%; recall:   0.00%; FB1:   0.00  2
CORREO_ELECTRONICO: precision:  94.57%; recall:  96.67%; FB1:  95.60  184
EDAD_SUJETO_ASISTENCIA: precision:  85.23%; recall:  90.72%; FB1:  87.89  413
FAMILIARES_SUJETO_ASISTENCIA: precision:  75.38%; recall:  67.12%; FB1:  71.01  65
           FECHAS: precision:  54.85%; recall:  92.74%; FB1:  68.93  908
         HOSPITAL: precision:  36.22%; recall:  46.46%; FB1:  40.71  127
 ID_ASEGURAMIENTO: precision:  56.85%; recall:  62.41%; FB1:  59.50  146
ID_CONTACTO_ASISTENCIAL: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
ID_EMPLEO_PERSONAL_SANITARIO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
ID_SUJETO_ASISTENCIA: precision:  58.82%; recall:   4.74%; FB1:   8.77  17
ID_TITULACION_PERSONAL

In [10]:

#Evaluate test results

with open('/content/gdrive/Team Drives/cis530/results/test_results.txt') as f:
    counts = evaluate(f)
report(counts)

processed 66137 tokens with 3522 phrases; found: 3904 phrases; correct: 2405.
accuracy:  96.94%; precision:  61.60%; recall:  68.29%; FB1:  64.77
            CALLE: precision:  16.18%; recall:  31.23%; FB1:  21.32  519
     CENTRO_SALUD: precision:  16.67%; recall:  33.33%; FB1:  22.22  6
CORREO_ELECTRONICO: precision:  89.33%; recall:  98.53%; FB1:  93.71  150
EDAD_SUJETO_ASISTENCIA: precision:  84.23%; recall:  92.00%; FB1:  87.94  355
FAMILIARES_SUJETO_ASISTENCIA: precision:  71.83%; recall:  59.30%; FB1:  64.97  71
           FECHAS: precision:  51.86%; recall:  97.67%; FB1:  67.74  727
         HOSPITAL: precision:  47.27%; recall:  56.52%; FB1:  51.49  110
 ID_ASEGURAMIENTO: precision:  46.67%; recall:  54.69%; FB1:  50.36  150
ID_CONTACTO_ASISTENCIAL: precision:   0.00%; recall:   0.00%; FB1:   0.00  1
ID_SUJETO_ASISTENCIA: precision:  33.33%; recall:   0.61%; FB1:   1.20  3
ID_TITULACION_PERSONAL_SANITARIO: precision:  72.73%; recall:  93.43%; FB1:  81.79  176
      INSTITUCION